In [1]:
import glob

import pandas as pd
from docx.api import Document

from tqdm import tqdm_notebook

import utils

In [24]:
# Read list of files
files = sorted(glob.glob('plan_obs/*.docx'))
for idx, file in enumerate(files):
    print(idx, file)

0 plan_obs\bsa_dkr_2014_4.docx
1 plan_obs\bsa_dkr_2015_1.docx
2 plan_obs\bsa_dkr_2019_2.docx
3 plan_obs\bsa_dkr_2019_3.docx


In [29]:
i = 1
document = Document(files[i])
table = document.tables[0]

In [30]:
data = []

keys = None
for i, row in tqdm_notebook(enumerate(table.rows)):
    text = (cell.text for cell in row.cells)

    if i == 0:
        keys = tuple(text)
        continue
    row_data = dict(zip(keys, text))
    data.append(row_data)

df = pd.DataFrame(data)
#df.to_csv('test.csv', sep=" ", header=True, index=False)

In [31]:
# How get element of df:
# iloc[row,columns]
# df.iloc[0,:].tolist() # [5,6,7,8]  - row index 0
# df.iloc[:,0].tolist() # [5,9,13,17]  - column index 0
# df.iloc[0,0] # 5  - cell(0,0)
# df.iloc[1:,2].tolist() # [11,15,19]  - column index 2, but skip first row# iloc[row,columns]
print(df.iloc[0,0])

In [32]:
df

,"Лосовский, Думский БСА,ЦАС авто+оператор","Лотова, Субаев ДКР в режиме интерферометра с базой 500м, 111 МГц, Оператор","Малофеев, Теплых, Малов О., Родин БСА АС-128, АС-32, ЦАС; ДКР, ЦП на 60 и 40 МГц Автомат, оператор","Потапов, Казанцев, Змеева, Пширков БСА,ЦАС, авто","Сулейманова БСА, ДКР, АС-128, ЦАС авто+оператор"
0,,10 объектов\nв промежутке 22h47m-00h55m:\n\n3С...,B0007+73\nБСА\nежедневно,J0007+7303 (B0007+73)\nКопии от Малофеева,
1,,,0054+66\nБСА\nежедневно,,
2,,10 объектов\nв промежутке 22h47m-00h55m\n*****...,,,
3,,,,"B0138+59,\n15-16,29-31.01\n15-16,26-28.02\n15-...",
4,,,"B0142+61,\nБСА\n01-14,17-28.01\n01-14,17-25.02...",,
5,,,B0205+64\nБСА\nежедневно,,
6,,,B0240+62\nБСА\nежедневно,,
7,,,,J0304+1932(B0301+19)\n20-31.01\n20-28.02\n20-3...,
8,,,,"B0320+39,\n15-16,29-31.01\n15-16,26-28.02\n15-...",
9,,,,"B0329+54,\n15-16,29-31.01\n15-16,26-28.02\n15-...",


Случай, которые требуется рассмотреть:

+ Даты указаны диапазоном
+ Даты указаны перечислением
+ Только копии
+ Копии + диапазон + перечисление
+ Ошибки в указании дат (исключение, обход)
+ Сохранение в формате имя пульсара и кортеж типа 'DD.MM.YYYY'

Выстаскиваем сначала копии, потом корректировка имени, затем все остальное

In [33]:
len(df)

69

In [34]:
# Создаем словарик "Имя основного наблюдателя" : индекс колонки
observers_idx = {}
for idx, value in enumerate(df.columns):
    head_obsr, *_ = value.split(',')
    head_obsr, *_ = head_obsr.split('\n')
    observers_idx[head_obsr] = idx

In [35]:
observers_idx

{'Лосовский': 0, 'Лотова': 1, 'Малофеев': 2, 'Потапов': 3, 'Сулейманова': 4}

In [36]:
def grab_copy_link(cell):
    # Забираем указание на копии. Удаляем их из массива
    copy_list = []
    for i in cell[:]:
        if 'копии' in i  or 'Копии' in i:
            copy_list.append(i)
            cell.remove(i)
        else:
            continue
    return cell, copy_list

In [37]:
def grab_name(cell):
    # Забираем имя пульсара. Удаляем его из массива
    name = ''
    for i in cell[:]:
        try:
            if i[0].isdigit():
                continue
            else:
                name += i
                cell.remove(i)
        except IndexError:  # обходим момент с пустой строкой
            continue
            
    return cell, name

In [38]:
def get_days(arr):
    res_arr = []
    for idx, value in enumerate(arr[:-1]):
        if '-' not in value:
            res_arr.append(value + '.' + arr[-1])
        else:
            first, second = value.split('-')
            res_arr.extend(["%.2d" % i + '.' + arr[-1] for i in range(int(first), int(second) + 1)])
    return res_arr
        

def get_obs(cell):
    full_obs = []
    # последний элемент в таком формате всегда месяц
    for idx, value in enumerate(cell):
        value = value.replace('.', ',')  # заменяем точки на запятые для разделения данных
        value = value.split(',')  # разделяем данные
        value = [i.replace(' ', '') for i in value if i] # убираем из выдачи пустые строки
        
        full_obs.extend(get_days(value))
    
    return full_obs

In [42]:
def grab_copy(copy_elem, i):
    # Распарсиваем копии пока что без учета того, что возможны сслыки на другие копии
    # import ipdb; ipdb.set_trace()
    *_, name_observer = copy_elem.split()  # в таком случае мы утверждаем, что элементов в строке три
    for key in observers_idx.keys():
        if round(utils.tanimoto(key, name_observer), 1) >= 0.6:
            obj = df.iloc[i][observers_idx[key]]
            cell = obj.split('\n')
            cell, name = grab_name(cell)
            obs = get_obs(cell)
        else:
            continue
    
    return name, obs

## Тестируем потоковую обработку строк

In [43]:
def parser_plan(head_obsr):
    idx_obsr = observers_idx[head_obsr]
    names_pulsars = []
    dict_obs = {}
    for i in range(len(df) - 1):
        obj = df.iloc[i][idx_obsr]
        cell = obj.split('\n')
        print(cell)
        cell, copy_list = grab_copy_link(cell) 
        if len(copy_list) == 0:
            cell, name = grab_name(cell)
            full_obs = get_obs(cell)
            names_pulsars.append(name)
            dict_obs[name] = full_obs
        else:
            for elem in copy_list:
                name, full_obs = grab_copy(elem, i)
                names_pulsars.append(name)
                dict_obs[name] = full_obs
                
    names_pulsars = [i for i in names_pulsars if i]
    return names_pulsars, dict_obs

In [44]:
head_obsr = 'Потапов'
n, d = parser_plan(head_obsr)

['J0007+7303 (B0007+73)', 'Копии от Малофеева']
['']
['']
['B0138+59,', '15-16,29-31.01', '15-16,26-28.02', '15-16,28-31.03']
['']
['']
['']
['J0304+1932(B0301+19)', '20-31.01', '20-28.02', '20-31.02', '']
['B0320+39,', '15-16,29-31.01', '15-16,26-28.02', '15-16,28-31.03']
['B0329+54,', '15-16,29-31.01', '15-16,26-28.02', '15-16,28-31.03']
['B0355+54,', '15-16,29-31.01', '15-16,26-28.02', '15-16,28-31.03']
['']
['']
['B0450+55', '20-31.01', '20-28.02', '20-31.02']
['']
['B0531+21', 'копия от Лосовского']
['']
['J0653+8051(0643+80)', 'копии от Малофеева']
['B0655+14', 'БСА', '08-12.01', '08-13.02', '08-13.03']
['']
['B0809+74', '20-31.01', '20-28.02', '20-31.02', 'При конфликтах приоритет программе Лосовского, копии от Лосовского']
['B0823+06,', '15-16,29-31.01', '15-16,26-28.02', '15-16,29-31.03']
['В0834+06', '20-31.01', '20-28.02', '20-31.03', 'При конфликтах приоритет программе Лосовского, копии от Лосовского']
['B0919+06', 'копии от Лосовского, Сулеймановой']
['']
['0943+10', 'копи

In [45]:
n

['B0007+73БСАежедневно',
 'B0138+59,',
 'J0304+1932(B0301+19)',
 'B0320+39,',
 'B0329+54,',
 'B0355+54,',
 'B0450+55',
 'B0531+21копия от Лосовского',
 'J0653+8051(0643+80)БСА',
 'B0655+14БСА',
 'B0823+06,',
 'В0834+06,',
 'B0919+06БСАежедневноПри конфликтах   приоритет программе Лосовского, копии от Лосовского',
 'БСАДКРежедневно',
 'J0953+0755(B0950+08)БСА+Копии от Потапова',
 'J1115+5030,(B1112+50)',
 'B1133+16',
 'J1239+2453 (B1237+25)ДКР',
 'B1508+55',
 'B1541+09,',
 'B1642-03,',
 'B1737+13,',
 'B1821+05,',
 'B1822-09,',
 'B1839+56,',
 'J1921+2153(B1919+21)',
 'J1929+1816J1932+1059(B1929+10)J1933+2421(B1931+24),B1937+21БСАежедневно',
 'B2016+28,',
 'B2020+28,',
 'БСАежедневно',
 'B2110+26,',
 'B2217+47БСАежедневноПри конфликтах — приоритет программе Лосовского',
 'B2224+65,',
 'B2303+30,']

In [17]:
d

{'': [],
 'B0007+73БСАежедневно': [],
 'B0138+59,': ['15.10',
  '16.10',
  '29.10',
  '30.10',
  '31.10',
  '15.11',
  '16.11',
  '29.11',
  '30.11',
  '15.12',
  '16.12',
  '28.12',
  '29.12',
  '30.12',
  '31.12'],
 'B0320+39,': ['15.10',
  '16.10',
  '29.10',
  '30.10',
  '31.10',
  '15.11',
  '16.11',
  '29.11',
  '30.11',
  '15.12',
  '16.12',
  '28.12',
  '29.12',
  '30.12',
  '31.12'],
 'B0329+54,': ['15.10',
  '16.10',
  '29.10',
  '30.10',
  '31.10',
  '15.11',
  '16.11',
  '29.11',
  '30.11',
  '15.12',
  '16.12',
  '28.12',
  '29.12',
  '30.12',
  '31.12'],
 'B0355+54,': ['15.10',
  '16.10',
  '29.10',
  '30.10',
  '31.10',
  '15.11',
  '16.11',
  '29.11',
  '30.11',
  '15.12',
  '16.12',
  '28.12',
  '29.12',
  '30.12',
  '31.12'],
 'B0450+55': ['20.10',
  '21.10',
  '22.10',
  '23.10',
  '24.10',
  '25.10',
  '26.10',
  '27.10',
  '28.10',
  '29.10',
  '30.10',
  '31.10',
  '20.11',
  '21.11',
  '22.11',
  '23.11',
  '24.11',
  '25.11',
  '26.11',
  '27.11',
  '28.11',
  '